# Create color ramps

## Converting from HSV or HSL to RGB

Why conversions among HSV, HSL, and RGB color systems? Because we want to create sequential or diverging color ramp sets off of a particular theme color - and a good way to do that is to keep the hue and use a sequence of different brightness or lightness. For this purpose, we need to convert an RGB hex into its corresponding HSV/HSL, tweak the V or L, before converting the new colors back to RGB hexes.

Below is the function that has `hue`, `saturation`, and `brightness`/`lightness` as input, and outputs the HEX of this color. The difference between the HSB and HSL systems is documented in this [Wikipedia page](https://en.wikipedia.org/wiki/HSL_and_HSV).
The algorithms of these conversions are found on [this website](https://www.rapidtables.com/convert/color/index.html). Note that HSB and HSV bear the same meaning.

In [ ]:
!curl -O https://raw.githubusercontent.com/Leejere/python-visualization-preset/main/convert_colors.ipynb

In [2]:
%run convert_colors.ipynb

## Creating monochromatic color ramps
In this section, let's try to make a functionality to create a **monochromatic color ramp** basing off of a theme color. This color ramp consists of a specific number of fading colors, the first being the theme color, and the last being white (#FFFFFF). Monochromatic, or sequential, color ramps, are used to demonstrate a values that vary only in extent (not in direction), where the theme color denotes the strongest or highest value, and the faded colors closer to the background color denote lower values. The background color (white or black) denotes zero.

We create this color ramp by changing only the *lightness* of the theme color. First, we calculate the HSL of the theme color, using the functions created in the last section. Then, we make new colors using the same hue and saturation, but with varying lightness values. On a white background, we increasing the lightness values until the color becomes white (denoting zero); on a black background, vice versa.

In [3]:
# A function that creates a monochromatic color ramp
# Takes 3 inputs: 1. theme color (string), 2. number of colors in this color ramp (int), 3. the lightness at the end of the sequence
# End lightness should be close to 100 if using white background, and 0 if using black background
# Outputs a list of the color ramp (in RGB HEX)

def create_mono_lightness_ramp(theme_color, num_of_colors, end_lightness = 100):
    
    # Get the HSL of the theme color
    theme_hue, theme_saturation, theme_lightness = hex_to_hsl(theme_color)[0], hex_to_hsl(theme_color)[1], hex_to_hsl(theme_color)[2]
    
    # Keep the H and S, make L lighter until 100% (or 0% against dark background)
    lightness_seq = np.linspace(theme_lightness, end_lightness, num_of_colors)
        
    hex_seq = []
    
    for i in range(num_of_colors):
        this_lightness = lightness_seq[i]
        this_hex = hsl_to_hex(theme_hue, theme_saturation, this_lightness)
        hex_seq.append(this_hex)
        
        
    hex_seq.reverse()
    
    return hex_seq

## Creating diverging color ramps
While monochromatic color ramps denote variations in extent, **diverging color ramps** denote variations not only in extent, but also in direction, e.g., positive and negative values. For this purpose, a diverging color ramp employs **two** theme colors, denoting the highest value in each direction. In this way, a diverging color ramp consists of two monochromatic color ramps, with the background color (white or black) presenting the zero value.

In [4]:
# A function that creates a diverging color ramp
# Takes 5 inputs:
# 1. Theme color 1
# 2. Theme color 2
# 3. number of colors based off of theme color 1
# 4. number of colors based off of theme color 2
# 5. end lightness based off of theme color 1
# 6. end lightness based off of theme color 2
# 7. insert a zero color in the middle

def create_div_lightness_ramp(theme_color_1, theme_color_2, num_of_colors_1, num_of_colors_2, end_lightness_1 = 100, end_lightness_2 = 100, insert_color = ''):
    
    # Get the HSL of the theme colors
    theme_hue_1, theme_saturation_1, theme_lightness_1 = hex_to_hsl(theme_color_1)[0], hex_to_hsl(theme_color_1)[1], hex_to_hsl(theme_color_1)[2]
    theme_hue_2, theme_saturation_2, theme_lightness_2 = hex_to_hsl(theme_color_2)[0], hex_to_hsl(theme_color_2)[1], hex_to_hsl(theme_color_2)[2]
    
    # If end_lightness_1 == end_lightness_2 == 0 or 100, 
    # it means they share a zero color, and the function automatically deletes a duplicate color
    
    ramp_1 = create_mono_lightness_ramp(theme_color_1, num_of_colors_1, end_lightness_1)
    ramp_2 = create_mono_lightness_ramp(theme_color_2, num_of_colors_2, end_lightness_2)
    ramp_1.reverse()
    ramp = ramp_1 + ramp_2
    
    if (end_lightness_1 == 100 & end_lightness_2 == 100) | (end_lightness_1 == 0 & end_lightness_2 == 0):
        ramp.pop(num_of_colors_1)
    if insert_color != '':
        ramp.insert(num_of_colors_1, insert_color)
    
    return ramp